# Stabilizatión of the simulation

In [2]:
## -----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Define important variables

temp = 9.5
pressure = 0.1

#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd("T_9.5_0.1_ramp.gsd",frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 3.0, tauP = 3.0, P = pressure);

#-----Write output and Run the Simulation

log_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.log"
gsd_file = "T_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

notice(2): Group "all" created containing 9000 particles
notice(2): constrain.rigid(): Removing all particles part of rigid bodies (except central particles).Particle tags may change.
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "rigid_center" created containing 1000 particles


In [3]:
hoomd.run(3e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:11 | Step 2224557 / 2524000 | TPS 55.6458 | ETA 01:29:41
Time 00:00:21 | Step 2225104 / 2524000 | TPS 54.6955 | ETA 01:31:04
Time 00:00:31 | Step 2225785 / 2524000 | TPS 67.9448 | ETA 01:13:09
Time 00:00:41 | Step 2226407 / 2524000 | TPS 62.093 | ETA 01:19:52
Time 00:00:51 | Step 2227026 / 2524000 | TPS 61.8717 | ETA 01:19:59
Time 00:01:01 | Step 2227566 / 2524000 | TPS 53.8621 | ETA 01:31:43
Time 00:01:11 | Step 2228165 / 2524000 | TPS 59.8421 | ETA 01:22:23
Time 00:01:21 | Step 2228761 / 2524000 | TPS 59.5249 | ETA 01:22:39
Time 00:01:31 | Step 2229430 / 2524000 | TPS 66.8899 | ETA 01:13:23
Time 00:01:41 | Step 2230035 / 2524000 | TPS 60.3479 | ETA 01:21:11
Time 00:01:51 | Step 2230639 / 2524000 | TPS 60.282 | ETA 01:21:06
Time 00:02:

**ERROR**: nlist.tree(): Particle 694 is out of bounds (x: 240.573, y: -357.293, z: 60.4497, fx: 2.6333, fy: -2.66833, fz:1.03604)


RuntimeError: Error updating neighborlist